## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-06-11-04-34 +0000,bbc,FBI fugitive found hiding in Wales can be extr...,https://www.bbc.com/news/articles/cn404z8g31mo...
1,2026-02-06-11-01-48 +0000,bbc,Head of firm founded by Mandelson to quit afte...,https://www.bbc.com/news/articles/cje1e3n8jgvo...
2,2026-02-06-11-00-21 +0000,nyt,TikTok’s ‘Addictive Design’ Found to Be Illega...,https://www.nytimes.com/2026/02/06/business/ti...
3,2026-02-06-11-00-19 +0000,wapo,NYC’s small landlords say they won’t survive M...,https://www.washingtonpost.com/nation/2026/02/...
4,2026-02-06-11-00-06 +0000,bbc,Three African country agrees to UK migrant ret...,https://www.bbc.com/news/articles/cp8204g8lejo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2408/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
177,trump,30
12,epstein,24
81,talks,14
122,new,14
65,ai,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
179,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...,93
188,2026-02-05-22-22-03 +0000,nypost,Hillary Clinton demands public hearing on Epst...,https://nypost.com/2026/02/05/us-news/hillary-...,77
126,2026-02-06-01-19-00 +0000,wsj,Trump Calls for New Nuclear Pact With Russia a...,https://www.wsj.com/politics/national-security...,75
153,2026-02-05-23-46-58 +0000,nyt,Bitcoin Drops to Lowest Price Since Trump Was ...,https://www.nytimes.com/2026/02/05/technology/...,70
207,2026-02-05-21-49-42 +0000,bbc,Trump's nuclear arms control push with Russia ...,https://www.bbc.com/news/articles/c70l27gy47do...,62


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
179,93,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...
53,62,2026-02-06-09-04-24 +0000,nyt,LA Olympics Chief Keeps Low Profile in Milan G...,https://www.nytimes.com/2026/02/05/world/europ...
101,56,2026-02-06-03-39-02 +0000,nypost,Top Senator calls for investigation into Mayor...,https://nypost.com/2026/02/05/us-news/senator-...
292,53,2026-02-05-17-21-57 +0000,bbc,Russia and Ukraine exchange prisoners as peace...,https://www.bbc.com/news/articles/c62np09q7l1o...
97,50,2026-02-06-04-26-26 +0000,nypost,AOC touts money for her NYC district in fundin...,https://nypost.com/2026/02/05/us-news/aoc-tout...
331,44,2026-02-05-15-27-00 +0000,wsj,The Bank of England and ECB kept interest rate...,https://www.wsj.com/economy/central-banking/ba...
63,44,2026-02-06-08-37-00 +0000,bbc,Amazon shares fall as it joins Big Tech AI spe...,https://www.bbc.com/news/articles/c150e144we3o...
153,40,2026-02-05-23-46-58 +0000,nyt,Bitcoin Drops to Lowest Price Since Trump Was ...,https://www.nytimes.com/2026/02/05/technology/...
131,37,2026-02-06-00-44-49 +0000,bbc,US police believe Savannah Guthrie's missing m...,https://www.bbc.com/news/articles/c2e18wvrx9xo...
156,36,2026-02-05-23-37-02 +0000,startribune,"In rare move, this northern Minnesota county i...",https://www.startribune.com/in-rare-move-this-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
